In [2]:
import pandas as pd  
import numpy as np
import json

path = '../kernels/'
def envir_values(env):
    '''
    return environment values according to generated data \ 
    env: 'smooth' or 'rough'
    '''
    # read reward data
    if env == 'smooth':
        file_name = 'kernel2.json'
    else:
        file_name = 'kernel1.json'
        
    with open(path+file_name) as file:
        # 2. 加载 JSON 数据
        data = json.load(file)
    
    # set list values to save environment reward structure
    env_nums = len(data)  # 40
    env_values = []   # final length is env_nums and it's a 2 dimensional vector
    for i in range(env_nums):
        per_values = []
        for _ in range(30):
            per_values.append(data[str(i)][str(_)]['y'])
        env_values.append(per_values)
    return env_values

In [2]:
import numpy as np
np.random.choice([5,7,8,9], 1, replace=False)

array([5])

In [54]:
envir_values('smooth')[0]

[0.6222820205861768,
 0.7077261147550069,
 0.7444234881705704,
 0.5918715728401014,
 0.2908811829456651,
 0.03493634213015373,
 0.0,
 0.24047814360533465,
 0.627919182869412,
 0.9224195494889865,
 1.0,
 0.9181612413089759,
 0.7635794406028203,
 0.6125632037580412,
 0.5643679317568718,
 0.6200652170459323,
 0.6118417283169145,
 0.4199793497051235,
 0.16885204124776204,
 0.050837115602884037,
 0.10078672838569698,
 0.2572986281251589,
 0.47390428036316756,
 0.6337679668456351,
 0.6195673901029147,
 0.5269544974715014,
 0.5130896635333806,
 0.5807534731301708,
 0.6594374222987237,
 0.6734604750685793]

In [51]:
# data 有 40个环境，每个环境有30个点
# give the examples
def show_example(smoothness:str) -> str:
    """return the example to the gpt

    Args:
        smoothness (str): the type of the environment
    """
    rows = 30
    envs = envir_values(smoothness)
    
    env_index1,env_index2,env_index3,env_index4 = np.random.randint(0,40,size=4)
    env1,env2,env3,env4 = envs[env_index1],envs[env_index2],envs[env_index3],envs[env_index4]
    envs = [env1,env2,env3,env4]
    example_data = [env_index1,env_index2,env_index3,env_index4]
    max1,max2,max3,max4 = np.random.randint(60,81,size=4)
    maxs = [max1,max2,max3,max4]
    example = ''
    for _ in range(len(envs)):
        current_env = ''
        env = envs[_]
        current_max = maxs[_]
        for i in range(rows):
            if i == 0:
                example += f'example{_} is:\n'
            current_env += f'{int(current_max*env[i])} '
            if i == 29:
                current_env += '\n\n'
                example += current_env
    example += 'If you understand the distribution, say Next'
    return example,example_data

In [69]:
a = '(1, 2, 3)'
eval(a) == (1,2,3)

True

In [31]:
a,b = show_example('smooth')
a

'example0 is:\n29 24 31 50 68 72 64 52 36 21 8 0 4 28 58 77 76 61 40 18 9 17 36 49 51 47 45 48 57 69 \n\nexample1 is:\n11 15 40 68 80 63 25 0 0 12 21 27 27 19 8 8 28 56 71 66 52 45 46 43 27 7 0 6 20 46 \n\nexample2 is:\n23 19 24 39 54 57 51 41 28 16 6 0 3 22 46 61 60 48 31 15 7 13 28 39 40 37 36 38 45 55 \n\nexample3 is:\n43 48 37 21 22 43 64 72 67 58 57 65 70 60 36 15 11 24 34 27 11 0 3 24 52 64 56 44 39 37 \n\nIf you understand the distribution, say Next'

In [2]:
import pandas as pd  
import numpy as np
import json
path = '../kernels/'
file_name = 'kernel2.json'

with open(path+file_name) as file:
    # 2. 加载 JSON 数据
    data = json.load(file)

In [47]:
np.random.permutation([5,10]).tolist()*5

[10, 5, 10, 5, 10, 5, 10, 5, 10, 5]

In [44]:
a = [5,10]
a*4

[5, 10, 5, 10, 5, 10, 5, 10]

In [ ]:
import numpy as np
import pandas as pd
import instruction as ins 
import configuration as conf
from openai import AzureOpenAI

# set the gpt4 agent
# configure the openai 

api_key ='ab5cdc60426f463d9192ae064e829a2f'
azure_endpoint = 'https://bishe.openai.azure.com/'
path = 'data/'

client = AzureOpenAI(
  api_key = api_key,  
  api_version = "2024-02-15-preview",
  azure_endpoint = azure_endpoint
)

# init the example background
smooth_envs = conf.envir_values('smooth')
rough_envs = conf.envir_values('rough')
env_type = ['smooth','rough']
xs = []
ys = []
zs = []
zs_scaled = []

# load instructions
pt = 'expinstructions/'
instructions_files = [pt+'example_eng.txt',pt+'experiment_eng.txt',pt+'change_eng.txt',pt+'finish_eng.txt']
instructions = []
for name in instructions_files:
  file = open(name, 'r',encoding='utf-8')
  instructions.append(file.read())
  file.close()

# temperature range
temperatures = [1]
horizions = [10,5]
tem_range = len(temperatures)
subs = 8; # number of subjects in each temperature

# formal study
for i in range(tem_range):
  for j in range(subs-4,subs):
    temperature = temperatures[i]
    new_env_type = np.random.permutation(env_type)
    print(new_env_type)
    if new_env_type[0] == 'smooth':
      env_order = [smooth_envs,rough_envs]
    else:
      env_order = [rough_envs,smooth_envs]
      
    first_examples = conf.show_example(new_env_type[0])
    second_examples = conf.show_example(new_env_type[1])
    examples = [first_examples,second_examples]

    first_number = conf.draw_numbers(first_examples[0],first_examples[1])[0]
    second_number = conf.draw_numbers(second_examples[0],second_examples[1])[0]
    numbers = [first_number,second_number]
    xs = []
    ys = []
    zs = []
    zs_scaled = []
    scores = 0
    # show the example
    content = instructions[0]
    print("U: " + instructions[0] + "\n" + "-" * 80 + "\n")
    # init the conversation
    response_content = "New Conversation!"
    conversation=[{"role": "system", "content": "you are my participant in the experiment, please follow my instructions to complete the experiment."}]
    conversation.append({"role":"user","content":content})
    max1 = conf.draw_max()
    max2 = conf.draw_max()
    maxs = [max1,max2]
    flag = 1
    trials = 0 
    round = 0 #0 for first and 1 for second
    horizions = 55

    while True:
        response = client.chat.completions.create(
            model="gpt4o", # use deployment name for Azure API
            messages=conversation,
            temperature=temperature
        )

        response_content = response.choices[0].message.content
        conversation.append({"role": "assistant", "content": response_content})
        print("A: " + response_content + "\n" + "-" * 80 + "\n")
        
        #show the example1
        if response_content == 'OK':
          example = examples[round][2]
          print("U: " + example + "\n" + "-" * 80 + "\n")
          conversation.append({"role":"user","content":example})
        
        #show the example2
        if  'NEXT' in response_content and flag == 1:
            flag = 0
            example = examples[round][3]
            print("U: " + example + "\n" + "-" * 80 + "\n")
            conversation.append({"role":"user","content":example})
        
        #understand the examples
        if 'NEXT' in response_content and flag == 0:
          #first round
          if round == 0:
            content = instructions[1]
            print("U: " + content + "\n" + "-" * 80 + "\n")
            conversation.append({"role":"user","content":content})
            flag = 1
          
        if response_content == 'START':
          #start experiment
          trials += 1
          x = np.random.randint(0,18)
          y = np.random.randint(0,18)
          index = 18*x+y
          z = env_order[round][numbers[round]][index]
          z_scaled = int(np.round(z*maxs[round]+np.random.randn()))
          scores += z_scaled
          xs.append(x)
          ys.append(y)
          zs.append(z)
          zs_scaled.append(z_scaled)
          if round == 0:
            content = f'Experiment start! You have {55-trials} trials left, randomly generated coordinate is [{x},{y}],its value is {z_scaled}, your sum of reveived values are {scores}。You are my participant,please give me your next choice just with the form of [x,y]'
          else:
            content = f'New type of environment experiment starts! You have {55-trials} trials left, randomly generated coordinate is [{x},{y}],its value is {z_scaled},your sum of reveived values are {scores}。You are my participant,please give me your next choice just with the form of [x,y]'            
          conversation.append({"role":"user","content":content})
          print("U: "+ content+ "\n" + "-" * 80 + "\n")
        
        if response_content[0] == '[':
          trials += 1
          choice = eval(response_content)
          x = choice[0]
          y = choice[1]
          index = 18*x+y
          z = env_order[round][numbers[round]][index]
          z_scaled = int(np.round(z*maxs[round]+np.random.randn()))
          scores += z_scaled
          xs.append(x)
          ys.append(y)
          zs.append(z)
          zs_scaled.append(z_scaled)
          content = f'You have {55-trials} trials left,you choose [{x},{y}],its value is {z_scaled}, your sum of reveived values are {scores}.Please give me your next choice just with the form of [x,y]'
          conversation.append({"role":"user","content":content})
          print("U: "+ content+ "\n" + "-" * 80 + "\n")

        if trials == horizions:
          if round == 0: #第一个环境
            round += 1
            trials = 0
            content = instructions[2]
            conversation.append({"role":"user","content":content})
            print("U: " + content + "\n" + "-" * 80 + "\n")
          else:
            content = instructions[3]
            conversation.append({"role":"user","content":content})
            print("U: " + content + "\n" + "-" * 80 + "\n")
        
        if 'FINISH' in response_content:
          break

    # save data
    choices = np.append(np.arange(horizions),np.arange(horizions))+1
    xs = np.array(xs)+1
    ys = np.array(ys)+1
    zs = np.array(zs)
    z_scaled = np.array(z_scaled)
    env_number = np.array([first_number]*horizions+[second_number]*horizions)
    global_max = np.array([max1]*horizions+[max2]*horizions)
    t = np.array([temperature]*horizions*2)
    env = np.array([new_env_type[0]]*horizions+[new_env_type[1]]*horizions)
    data = pd.DataFrame({'trials':choices,'x':xs,'y':ys,'zs':zs,'z_scaled':zs_scaled,'number':env_number,'globalmax':global_max,'env_type':env,'t':t})
    data.to_csv(path+f'sub{j}_t{temperature}.csv')

    # save strategy
    with open('strategy/'+f"sub{j}_t{temperature}.txt", "w",encoding='utf-8') as file:
      file.write(response_content)
      file.close()
  

In [48]:
np.random.randint(0,2)

0

In [50]:
eval('[0,1,2]')

[0, 1, 2]

In [70]:
import numpy as np
import pandas as pd
import load as ld
from openai import AzureOpenAI

# set the gpt4 agent
# configure the openai 

api_key ='ab5cdc60426f463d9192ae064e829a2f'
azure_endpoint = 'https://bishe.openai.azure.com/'
path = 'data/'

client = AzureOpenAI(
  api_key = api_key,  
  api_version = "2024-02-15-preview",
  azure_endpoint = azure_endpoint
)

# init the example background
smooth_envs = ld.envir_values('smooth')
rough_envs = ld.envir_values('rough')


env_type = ['smooth','rough']
horizons = [5,10]
xs = []
ys = []
zs = []
zs_scaled = []

pt = '../exp_instructions/'
instructions_files = [pt+'experiment_eng.txt',pt+'test_eng.txt',pt+'right_eng.txt',pt+'finish_eng.txt']
instructions = []
for name in instructions_files:
  file = open(name, 'r',encoding='utf-8')
  instructions.append(file.read())
  file.close()

# temperature range
temperatures = [1]
tem_range = len(temperatures)
subs = 8; # number of subjects in each temperature

# formal study
for i in range(tem_range):
  for j in range(1):
    # special agent
    temperature = temperatures[i]
    horizons_order = np.random.permutation(horizons).tolist()
    agent_env = env_type[np.random.randint(0,2)]
    env_values = ld.envir_values(agent_env) # 40 environments
    examples,example_index = ld.show_example(agent_env)
    experiment_index = ld.draw_numbers(example_index)
    maxs = ld.draw_max(len(experiment_index))
    xs = []
    ys = []
    zs = []
    zs_scaled = []
    scores = 0
    flag = 0
    # show the example
    content = instructions[0]
    print("U: " + instructions[0] + "\n" + "-" * 80 + "\n")
    # init the conversation
    response_content = "New Conversation!"
    conversation=[{"role": "system", "content": "you are my participant in the experiment, please follow my instructions to complete the experiment."}]
    conversation.append({"role":"user","content":content})

    for k in range(len(horizons_order)):
      # in the given environment
      current_horzion = horizons_order[k]
      current_env = env_values[experiment_index[k]]
      trials = 0 

      while True:
        if flag == 0:
            response = client.chat.completions.create(
                model="gpt4o", # use deployment name for Azure API
                messages=conversation,
                temperature=temperature
            )

            response_content = response.choices[0].message.content
            conversation.append({"role": "assistant", "content": response_content})
        elif flag == 1:
            response_content = 'CONTINUE'
            flag = 0

        print("A: " + response_content + "\n" + "-" * 80 + "\n")
        
        #show the examples
        if response_content == 'OK':
            conversation.append({"role":"user","content":examples})
            print("A: " + examples + "\n" + "-" * 80 + "\n")
        
        #understand the examples
        if 'NEXT' in response_content:
            content = instructions[1]
            conversation.append({"role":"user","content":content})
            print("U: " + content + "\n" + "-" * 80 + "\n")
        
        # judge whether it is right
        if response_content[0] == '(':
           if len(eval(response_content)) == 3:
              if eval(response_content) == (1,1,3):
                 content = instructions[2]
                 print("U: " + content + "\n" + "-" * 80 + "\n")
                 conversation.append({"role":"user","content":content})
              else:
                 content = 'some answers are wrong. Give me the right answers with the form of (X,Y,Z)'
                 print("U: " + content + "\n" + "-" * 80 + "\n")
                 conversation.append({"role":"user","content":content})

        if response_content[0] == '[':
            trials += 1
            choice = eval(response_content)
            x = choice[0]
            z = current_env[x]
            z_scaled = int(np.round(z*maxs[k]+np.random.randn()))
            scores += z_scaled
            scores += z_scaled
            xs.append(x)
            zs.append(z)
            zs_scaled.append(z_scaled)
            content = f'You have {current_horizon-trials} trials left,you choose [{x}],its value is {z_scaled}, your sum of reveived values are {scores}.Please just give me your next choice just with the form of [x]'
            conversation.append({"role":"user","content":content})
            print("U: "+ content+ "\n" + "-" * 80 + "\n")        

        if response_content == 'START':
            #start experiment
            trials += 1
            x = np.random.randint(0,30)
            z = current_env[x]
            z_scaled = int(np.round(z*maxs[k]+np.random.randn()))
            scores += z_scaled
            xs.append(x)
            zs.append(z)
            zs_scaled.append(z_scaled)
            if k == 0:
                content = f'Experiment start! You have {current_horzion-trials} trials left, randomly generated coordinate is [{x}],its value is {z_scaled}, your sum of reveived values are {scores}。You are my participant,please give me your next choice just with the form of [x]'
            else:
                content = f'A new environment experiment starts! You have {current_horzion-trials} trials left, randomly generated coordinate is [{x}],its value is {z_scaled},your sum of reveived values are {scores}。You are my participant,please give me your next choice just with the form of [x]'            
            conversation.append({"role":"user","content":content})
            print("U: "+ content+ "\n" + "-" * 80 + "\n")
        


        if response_content == 'CONTINUE':
           if trials != 0:
                flag = 1
                break
           else:
                #start experiment
                trials += 1
                x = np.random.randint(0,30)
                z = current_env[x]
                z_scaled = int(np.round(z*maxs[k]+np.random.randn()))
                scores += z_scaled
                xs.append(x)
                zs.append(z)
                zs_scaled.append(z_scaled)
                if k == 0:
                    content = f'Experiment start! You have {current_horzion-trials} trials left, randomly generated coordinate is [{x}],its value is {z_scaled}, your sum of reveived values are {scores}。You are my participant,please just give me your next choice with the form of [x]'
                else:
                    content = f'A new environment experiment starts! You have {current_horzion-trials} trials left, randomly generated coordinate is [{x}],its value is {z_scaled},your sum of reveived values are {scores}。You are my participant,please just give me your next choice just with the form of [x]'            
                conversation.append({"role":"user","content":content})
                print("U: "+ content+ "\n" + "-" * 80 + "\n")
        
    # save data
    choices = []
    env_numbers = []
    agent_horizons = []
    globalmax = []
    t = []
    for p in range(len(horizons_order)):
        current_horizon = horizons_order[p]
        env_number = experiment_index[p]
        maxi = maxs[p]
        for q in range(current_horizon):
           choices.append(q+1)
           env_numbers.append(env_number)
           agent_horizons.append(current_horizon)
           globalmax.append(maxi)
           t.append(temperature)
           

    xs = np.array(xs)+1
    zs = np.array(zs)
    z_scaled = np.array(z_scaled)
    env_numbers = np.array(env_numbers)
    global_max = np.array(globalmax)
    t = np.array(t)
    data = pd.DataFrame({'trials':choices,'x':xs,'zs':zs,'z_scaled':zs_scaled,'kernel':env_number,'globalmax':global_max,'t':t})
    data['environment'] = agent_env
    data.to_csv(path+f'sub{j}_t{temperature}.csv')

    # save strategy
    with open('strategy/'+f"sub{j}_t{temperature}.txt", "w",encoding='utf-8') as file:
        file.write(response_content)
        file.close()

U: In the following study, you will be presented with a series of 8 different environments to explore, depicted as a row of boxes. By choosing any of the boxes, you will earn points associated with each unique box. For each row of boxes, you will have either 5 or 10 choices, with the number of remaining choices in my prompt. When you run out of choices, you will start a new trial on the next unexplored environment.

Each environment starts with a single box revealed its value. Just give me your choice with the coordinate form of [x], and I  will give you corresponding to the number of points you gain. Previously revealed boxes can also be reselected, although there may be small changes in the point value.

It is your task to gain as many points as possible across all 8 environments. 

Important! Points are clustered along the row of boxes, such that boxes with high-value points tend to appear close to each other and boxes with low-value points tend to appear close to each other. All pa

NameError: name 'current_horizon' is not defined

In [72]:
y = 5
x = 4
horizons = [5,10]
horizons_order = np.random.permutation(horizons).tolist()
